# Luke McMeans - Finance API Homework

**TASK 1: Fetch Basic Stock Data**

In [1]:
# api key
api_key = ""

In [2]:
import pandas as pd
import requests

# get inputted stocks, as many until they decide to quit
idx = 1
print("Input at least 2 stocks (type 'q' to quit)")
stocks = []
cont = True
while(cont):
    stock = input(f"Input Stock {idx}: ")
    idx += 1
    if stock == 'q':
        break
    stocks.append(stock)

# in case no stocks were inputted, give a sample
if not stocks:
    stocks = ['AAPL', 'MSFT', 'MSTR']

Input at least 2 stocks (type 'q' to quit)


In [3]:
# convert to query string
stock_str = ",".join(stocks)
query_string = {'symbols': stock_str}

# fetch data
url = "https://yfapi.net/v6/finance/quote"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [4]:
# convert to dataframe, and translate market price to 
df = pd.DataFrame(response.json()['quoteResponse']['result'])

# function to translate columns to proper dollar strings
def convert_to_dollar(col):
    df[col] = df[col].apply(lambda x: f"${x:,.2f}" if pd.notna(x) else "N/A")

convert_to_dollar('regularMarketPrice')

# print statements for all rows
for _, row in df.iterrows():
    ticker = row['symbol']
    name = row['shortName']
    market_price = row['regularMarketPrice']
    
    print(f"Stock Ticker: {ticker}, Company: {name}, Current Market Price: {market_price}")

df

Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36
Stock Ticker: MSFT, Company: Microsoft Corporation, Current Market Price: $399.73
Stock Ticker: MSTR, Company: MicroStrategy Incorporated, Current Market Price: $263.27


,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,shortName,longName,...,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,bid,ask,bidSize,askSize,fullExchangeName,displayName,symbol
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,Apple Inc.,Apple Inc.,...,239.130,43602013,247.04,240.24,240.34,1,1,NasdaqGS,Apple,AAPL
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,Microsoft Corporation,Microsoft Corporation,...,394.245,19488244,397.90,399.70,399.96,1,1,NasdaqGS,Microsoft,MSFT
2,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,MicroStrategy Incorporated,MicroStrategy Incorporated,...,244.010,19161325,250.51,258.30,263.68,4,2,NasdaqGS,NaN,MSTR


**TASK 2.1: Fetch Additional Data Using Modules**

Note: The 52WeekHigh, 52WeekLow, and ReturnOnAssests are found in the `summaryDetail` and `financialData` modules, so they are automatically retrieved. Other modules can be selected and turned into it's own dataframe.

In [37]:
# get inputted modules, as many until they decide to quit
idx = 1
print("Input modules you wan't to find (type 'q' to stop)")
print("OPTIONS: assetProfile, fundProfile, defaultKeyStatistics, calendarEvents, \n \
      incomeStatementHistory, incomeStatementHistoryQuarterly, cashflowStatementHistory, \n \
      earnings, earningsHistory, insiderHolders, balanceSheetHistory, cashflowStatementHistory, \n \
      cashflowStatementHistoryQuarterly, insiderTransactions, secFilings, indexTrend, \n \
      sectorTrend, earningsTrend, netSharePurchaseActivity, upgradeDowngradeHistory,\n \
      institutionOwnership, recommendationTrend, balanceSheetHistory, balanceSheetHistoryQuarterly,\n \
      fundOwnership, majorDirectHolders, majorHoldersBreakdown, price, quoteType, esgScores")
modules = ['summaryDetail', 'financialData']
cont = True
while(cont):
    mod = input(f"Input Module {idx}: ")
    idx += 1
    if mod == 'q':
        break
    modules.append(mod)

Input modules you wan't to find (type 'q' to stop)
OPTIONS: assetProfile, fundProfile, defaultKeyStatistics, calendarEvents, 
       incomeStatementHistory, incomeStatementHistoryQuarterly, cashflowStatementHistory, 
       earnings, earningsHistory, insiderHolders, balanceSheetHistory, cashflowStatementHistory, 
       cashflowStatementHistoryQuarterly, insiderTransactions, secFilings, indexTrend, 
       sectorTrend, earningsTrend, netSharePurchaseActivity, upgradeDowngradeHistory,
       institutionOwnership, recommendationTrend, balanceSheetHistory, balanceSheetHistoryQuarterly,
       fundOwnership, majorDirectHolders, majorHoldersBreakdown, price, quoteType, esgScores


In [38]:
# let's use the first inputted stock for reference
stock = stocks[0]

# convert to query string
module_str = ",".join(modules)
query_string = {
    'symbols': stock,
    'modules': module_str
}

# fetch data
url = f"https://yfapi.net/v11/finance/quoteSummary/{stock}"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [41]:
result = response.json()['quoteSummary']['result']

# retrieve dataframes 
stats_dfs = []
for key, value in result[0].items():
    print(f"\nDataframe for module: {key}")
    # for all values not summaryDetail and financialData
    if key != 'summaryDetail' and key != 'financialData':
        # find right format
        if isinstance(value, dict):
            for k, v in value.items():
                if isinstance(v, list):
                    df = pd.DataFrame(v)
                    break
            else:
                df = pd.DataFrame([value])
        elif isinstance(value, list):
            df = pd.DataFrame(value)
        else:
            df = pd.DataFrame([value])
        
        # only get 'raw' for the dictionaries that apply
        for column in df.columns:
            df[column] = df[column].apply(lambda x: x['raw'] if isinstance(x, dict) and 'raw' in x else x)
    else:
        df = pd.DataFrame(value)
        stats_dfs.append(df)
        
    # print df
    display(df)

# print 

# find and print desired stats for this portion
stats_dfs[0] = stats_dfs[0].loc['fmt']
stats_dfs[1] = stats_dfs[1].loc['fmt']
weekHigh = stats_dfs[0]['fiftyTwoWeekHigh']
weekLow = stats_dfs[0]['fiftyTwoWeekLow']
roa = stats_dfs[1]['returnOnAssets']
print(f"\n52 Week High: {weekHigh} \n52 Week Low: {weekLow} \nReturn on Assets: {roa}")


Dataframe for module: summaryDetail


,maxAge,priceHint,previousClose,open,dayLow,dayHigh,regularMarketPreviousClose,regularMarketOpen,regularMarketDayLow,regularMarketDayHigh,...,toCurrency,lastMarket,coinMarketCapLink,volume24Hr,volumeAllCurrencies,circulatingSupply,algorithm,maxSupply,startDate,tradeable
raw,1,2,247.04,244.33,239.13,244.98,247.04,244.33,239.13,244.98,...,None,None,None,NaN,NaN,NaN,None,NaN,NaN,False
fmt,1,2,247.04,244.33,239.13,244.98,247.04,244.33,239.13,244.98,...,None,None,None,NaN,NaN,NaN,None,NaN,NaN,False
longFmt,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,NaN,NaN,NaN,None,NaN,NaN,False



Dataframe for module: insiderHolders


,maxAge,name,relation,url,transactionDescription,latestTransDate,positionDirect,positionDirectDate,positionIndirect,positionIndirectDate
0,1,ADAMS KATHERINE L,General Counsel,,Stock Gift,1730764800,179043.0,1.730765e+09,NaN,NaN
1,1,AUSTIN WANDA M,Director,,Conversion of Exercise of derivative security,1738281600,1588.0,1.738282e+09,NaN,NaN
2,1,GORSKY ALEX,Director,,Conversion of Exercise of derivative security,1738281600,5539.0,1.738282e+09,NaN,NaN
3,1,JUNG ANDREA,Director,,Conversion of Exercise of derivative security,1738281600,76005.0,1.738282e+09,NaN,NaN
4,1,KONDO CHRISTOPHER,Officer,,Sale,1731888000,15419.0,1.731888e+09,NaN,NaN
5,1,LEVINSON ARTHUR D,Director,,Sale,1738540800,4215580.0,1.738541e+09,NaN,NaN
6,1,LOZANO MONICA C.,Director,,Conversion of Exercise of derivative security,1738281600,8607.0,1.738282e+09,NaN,NaN
7,1,SUGAR RONALD D,Director,,Conversion of Exercise of derivative security,1738281600,109311.0,1.738282e+09,NaN,NaN
8,1,WAGNER SUSAN L,Director,,Conversion of Exercise of derivative security,1738281600,68533.0,1.738282e+09,NaN,NaN
9,1,WILLIAMS JEFFREY E,Chief Operating Officer,,Sale,1734307200,NaN,NaN,389944.0,1.734307e+09



Dataframe for module: price


,maxAge,preMarketChangePercent,preMarketChange,preMarketTime,preMarketPrice,preMarketSource,postMarketChangePercent,postMarketChange,postMarketTime,postMarketPrice,...,currency,quoteSourceName,currencySymbol,fromCurrency,toCurrency,lastMarket,volume24Hr,volumeAllCurrencies,circulatingSupply,marketCap
0,1,-0.011334,-2.79999,1740580199,244.24,FREE_REALTIME,-0.000541,-0.130005,1740611300,240.23,...,USD,Nasdaq Real Time Price,$,None,None,None,{},{},{},3610712211456



Dataframe for module: financialData


,maxAge,currentPrice,targetHighPrice,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationMean,recommendationKey,numberOfAnalystOpinions,totalCash,...,grossProfits,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,profitMargins,financialCurrency
raw,86400,240.36,325.0,197.0,252.22575,257.5,2.06522,buy,40,53774999552,...,184102993920,93833871360,108293996544,0.101,0.04,0.46519,0.34706,0.34459,0.24295,USD
fmt,86400,240.36,325.00,197.00,252.23,257.50,2.07,buy,40,53.77B,...,184.1B,93.83B,108.29B,10.10%,4.00%,46.52%,34.71%,34.46%,24.30%,USD
longFmt,86400,NaN,NaN,NaN,NaN,NaN,NaN,buy,40,"53,774,999,552",...,"184,102,993,920","93,833,871,360","108,293,996,544",NaN,NaN,NaN,NaN,NaN,NaN,USD



52 Week High: 260.10 
52 Week Low: 164.08 
Return on Assets: 22.52%


**TASK 2.2: Fetch Additional Data Using Modules**

In [45]:
# get trending/popular stocks
url = "https://yfapi.net/v1/finance/trending/US"  # US market trending stocks
headers = {
    'x-api-key': api_key
}
response = requests.get(url, headers=headers)
trending_data = response.json()

# turn the trending stocks into array
stocks = []
for quote in trending_data['finance']['result'][0]['quotes']:
    stocks.append(quote['symbol'])

In [46]:
# convert to query string
stock_str = ",".join(stocks)
query_string = {'symbols': stock_str}

# fetch data
url = "https://yfapi.net/v6/finance/quote"
headers = {
    'x-api-key': api_key
}
response = requests.request("GET", url, headers=headers, params=query_string)

In [47]:
# convert to dataframe, and translate market price to 
df = pd.DataFrame(response.json()['quoteResponse']['result'])

# convert columns to dolar value
convert_to_dollar('regularMarketPrice')
convert_to_dollar('fiftyTwoWeekHigh')
convert_to_dollar('fiftyTwoWeekLow')

print("Trending Stocks:")
df[['symbol', 'shortName', 'regularMarketPrice', 'fiftyTwoWeekHigh', 'fiftyTwoWeekLow']]

Trending Stocks:


,symbol,shortName,regularMarketPrice,fiftyTwoWeekHigh,fiftyTwoWeekLow
0,NVDA,NVIDIA Corporation,$131.28,$153.13,$75.61
1,CRM,"Salesforce, Inc.",$307.33,$369.00,$212.00
2,SNOW,Snowflake Inc.,$166.19,$233.88,$107.13
3,IONQ,"IonQ, Inc.",$29.93,$54.74,$6.22
4,BYND,"Beyond Meat, Inc.",$3.56,$12.12,$3.30
5,APP,Applovin Corporation,$331.00,$525.15,$57.40
6,EBAY,eBay Inc.,$69.14,$71.61,$46.53
7,AI,"C3.ai, Inc.",$26.44,$45.08,$18.85
8,NIO,NIO Inc.,$4.72,$7.71,$3.61
9,SNPS,"Synopsys, Inc.",$470.46,$624.80,$457.00
